In [1]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np

In [2]:
from proc_v1 import p
from ml_v1 import X_cat, X_num, X_all, target, kf, ss, config
from ml_v1 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter

In [3]:
sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform('data/train.csv').assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform('data/test.csv')
len(df_train)

750000

# 1. Polynomial Feature Generation & Selection

- 1차, 1 / 1차, 2차, 1차 / 1차, 2차 / 1차, 1차 / 2차, 3차 속성을 만듭니다.
- 전체 속성으로 선형 모델을 학습 시키고, 회귀계수를 봅니다.
- 다중 공선성으로 인해 회귀계수가 커진 케이스가 확인 됐습니다. VIF를 측정하여 속성을 선별합니다.

In [4]:
from itertools import combinations, product

X_r = [i + '_r' for i in X_num]
X_o2 = [(i + '_2' if i == j else i + '_mul_'+ j, (i, j)) for i, j in product(X_num, X_num) if i <= j]
X_1d1 = [(i + '_div_' + j, (i, j)) for i, j in product(X_num, X_num) if i != j]
X_o2d1 = [
    (i + '_div_' + j, (i, j)) for (i, a), j in product(X_o2, X_num) if j not in a 
]
X_o1d2 = [
    (i + '_div_' + j, (i, j)) for i, (j, a) in product(X_num, X_o2) if i not in a 
]

X_o3 = [
    (
        i + '_3' if i == j and j == k else i + '_2_mul_' + k if i == j else i + '_2_mul_' + k if i == k  else j + '_2_mul_' + i if j == k else i + '_mul_' + j + '_mul_' + k,
        (i, j, k)
    )
    for i, j, k in product(X_num, X_num, X_num) if i <= j and i <= k and j <= k
]

In [5]:
var_list = [(1 / df_train[a]).rename(a + '_r') for a in X_num]
var_list.extend([(df_train[a].astype('float32') * df_train[b].astype('float32')).rename(i) for i, (a, b) in X_o2])
var_list.extend([(df_train[a].astype('float32') / df_train[b].astype('float32')).rename(i) for i, (a, b) in X_1d1])
var_list.extend([(df_train[a].astype('float32') * df_train[b].astype('float32') * df_train[c].astype('float32')).rename(i) for i, (a, b, c) in X_o3])

df_sel = dproc.join_and_assign(
    df_train,
    pd.concat(var_list, axis = 1)
)
df_sel = dproc.join_and_assign(
    df_sel,
    pd.concat([(df_sel[a] / df_sel[b]).rename(i) for i, (a, b) in X_o2d1 + X_o1d2], axis = 1)
)
df_sel.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Calories_Log,Age_r,...,Body_Temp_div_Height_2,Body_Temp_div_Height_mul_Weight,Body_Temp_div_Weight_2,Body_Temp_div_Duration_mul_Height,Body_Temp_div_Duration_mul_Weight,Body_Temp_div_Duration_2,Body_Temp_div_Duration_mul_Heart_Rate,Body_Temp_div_Heart_Rate_mul_Height,Body_Temp_div_Heart_Rate_mul_Weight,Body_Temp_div_Heart_Rate_2
id,,,,,,,,,,,,,,,,,,,,,
0,True,36,189.0,82.0,26.0,101.0,41.000000,150.0,5.017280,0.027778,...,0.001148,0.002646,0.006098,0.008344,0.019231,0.060651,0.015613,0.002148,0.004950,0.004019
1,False,64,163.0,60.0,8.0,85.0,39.700001,34.0,3.555348,0.015625,...,0.001494,0.004059,0.011028,0.030445,0.082708,0.620313,0.058382,0.002865,0.007784,0.005495
2,False,51,161.0,64.0,7.0,84.0,39.799999,29.0,3.401197,0.019608,...,0.001535,0.003863,0.009717,0.035315,0.088839,0.812245,0.067687,0.002943,0.007403,0.005641
3,True,20,192.0,90.0,25.0,105.0,40.700001,140.0,4.948760,0.050000,...,0.001104,0.002355,0.005025,0.008479,0.018089,0.065120,0.015505,0.002019,0.004307,0.003692
4,False,38,166.0,61.0,25.0,102.0,40.599998,146.0,4.990433,0.026316,...,0.001473,0.004009,0.010911,0.009783,0.026623,0.064960,0.015922,0.002398,0.006525,0.003902


In [6]:
X_sel = [i for i in df_sel.columns if i not in ('Calories', 'Calories_Log')]

In [8]:
result = sgml.cv(df_sel, ss, {'X_std': X_sel}, config, lr_adapter, result_proc = [sgml.lr_learning_result])
result['valid_scores']

[0.06812648703351626]

In [9]:
result['model_result'][0]['coef'].sort_values()

std__Age_mul_Body_Temp      -2679.302825
std__Weight_div_Body_Temp   -2300.629808
std__Heart_Rate_2           -2001.656839
std__Age_div_Body_Temp      -1875.567715
std__Body_Temp_mul_Weight   -1743.840844
                                ...     
std__Height_mul_Weight       2120.526621
std__Duration_div_Height     2483.858796
std__Duration_mul_Height     2900.983539
std__Age                     3205.103881
std__Weight                  3328.888549
Length: 300, dtype: float64

In [10]:
result = sgml.cv(df_sel.sample(frac=0.1), ss, {'X_std': X_sel}, config, lr_adapter, result_proc = [sgml.lr_learning_result])
result['valid_scores']

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
from tqdm.notebook import tqdm

In [22]:
s_score1 = pd.Series({
    i: sgml.cv(df_sel, ss, {'X_std': [i]}, {**config, 'progress_callback': None}, lr_adapter, result_proc = [sgml.lr_learning_result])['valid_scores'][0]
    for i in tqdm(X_sel)
})
s_score1.sort_values(ascending = False).iloc[:50]

  0%|          | 0/300 [00:00<?, ?it/s]

Body_Temp_2_div_Age                 [0.9630523324012756]
Age_div_Body_Temp_2                 [0.9630053639411926]
Age_2_div_Heart_Rate                [0.9629924297332764]
Weight_div_Height                   [0.9629755616188049]
Weight_3                             [0.962936282157898]
Weight_2_div_Height                 [0.9629279971122742]
Height_div_Weight                   [0.9629268050193787]
Height_2_div_Weight                 [0.9628967046737671]
Weight_2                            [0.9628810286521912]
Weight_div_Height_2                 [0.9628792405128479]
Heart_Rate_div_Age_2                [0.9628680348396301]
Weight_2_mul_Height                 [0.9628543257713318]
Weight                              [0.9628183841705322]
Height_div_Weight_2                 [0.9627500772476196]
Height_mul_Weight                   [0.9627408981323242]
Height_2_mul_Weight                 [0.9627084732055664]
Weight_r                            [0.9626862406730652]
Sex                            

In [28]:
s_score1.apply(lambda x: x[0]).sort_values().iloc[:10]

Body_Temp_3                              0.291039
Body_Temp_2                              0.291284
Body_Temp                                0.291960
Body_Temp_r                              0.294626
Duration_div_Body_Temp_2                 0.317725
Duration_div_Body_Temp_mul_Heart_Rate    0.319305
Duration_div_Body_Temp                   0.322041
Duration_div_Heart_Rate                  0.322106
Body_Temp_mul_Duration_div_Heart_Rate    0.325333
Duration                                 0.326600
dtype: float64

In [26]:
df_sel['Body_Temp_3']

id
0         68921.000000
1         62570.777344
2         63044.789062
3         67419.148438
4         66923.406250
              ...     
749995    68417.937500
749996    66430.125000
749997    68417.937500
749998    65450.824219
749999    66923.406250
Name: Body_Temp_3, Length: 750000, dtype: float32

In [27]:
df_sel['Body_Temp_2']

id
0         1681.000000
1         1576.090088
2         1584.039917
3         1656.490112
4         1648.359863
             ...     
749995    1672.810181
749996    1640.250000
749997    1672.810181
749998    1624.089966
749999    1648.359863
Name: Body_Temp_2, Length: 750000, dtype: float32